In [1]:
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout
from keras.callbacks import EarlyStopping

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28, 28, 1), x_test.reshape(-1, 28, 28, 1)

c:\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def CNN_model(ac='sigmoid'):
    
    cnn = Sequential()
    
    cnn.add(Conv2D(32, [5, 5], input_shape=(28, 28, 1), activation=ac, padding='same', name='Conv2d_5x5_1'))

    cnn.add(Conv2D(64, [3, 3], activation=ac, padding='same', name='Conv2d_3x3_1'))
    cnn.add(MaxPooling2D([3, 3], name='maxpool_1'))

    cnn.add(Conv2D(128, [3, 3], activation=ac, padding='same', name='Conv2d_3x3_2'))
    cnn.add(MaxPooling2D([2, 2], name='maxpool_2'))

    cnn.add(Conv2D(128, [3, 3], activation=ac, padding='valid', name='Conv2d_3x3_3'))

    cnn.add(Flatten())
    cnn.add(Dense(10, activation='softmax'))

    return cnn

In [ ]:
activations = ['sigmoid', 'relu']
ac_logger = []
print("Activation compare:")
print("\tSigmoid vs ReLU\n")
for activation in activations:
    print("Activation Use", activation)
    model = CNN_model(ac=activation)
    model.compile(optimizer='sgd',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    callback = EarlyStopping(monitor='acc', min_delta=0.001, patience=5)
    ac_logger.append(model.fit(x_train, y_train, epochs=30, validation_split=0.1, callbacks=[callback]))
    eva = model.evaluate(x_test, y_test)
    print("Test accuracy: %.2f loss: %.2f" % (eva[1]*100, eva[0]))
    
    K.clear_session()

Activation compare:
	Sigmoid vs ReLU

Activation Use sigmoid
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2d_5x5_1 (Conv2D)        (None, 28, 28, 32)        832       
_________________________________________________________________
Conv2d_3x3_1 (Conv2D)        (None, 28, 28, 64)        18496     
_________________________________________________________________
maxpool_1 (MaxPooling2D)     (None, 9, 9, 64)          0         
_________________________________________________________________
Conv2d_3x3_2 (Conv2D)        (None, 9, 9, 128)         73856     
_________________________________________________________________
maxpool_2 (MaxPooling2D)     (None, 4, 4, 128)         0         
_________________________________________________________________
Conv2d_3x3_3 (Conv2D)        (None, 2, 2, 128)         147584    
_________________________________________________________________
flatten_1 (Flat

In [ ]:
color = ['r', 'b']
label_n = []
for idx, log in enumerate(ac_logger):
    label_n.append(activations[idx]+' acc')
    label_n.append(activations[idx]+' val_acc')
    plt.plot(log.history['acc'], '{}-.'.format(color[idx]))
    plt.plot(log.history['val_acc'], '{}--'.format(color[idx]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()

label_n = []
for idx, log in enumerate(ac_logger):
    label_n.append(activations[idx]+' loss')
    label_n.append(activations[idx]+' val_loss')
    plt.plot(log.history['loss'], '{}-.'.format(color[idx]))
    plt.plot(log.history['val_loss'], '{}--'.format(color[idx]))

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()

In [ ]:
activation = 'relu'
bs_logger = []

Batch_Sizes = [32, 64, 128, 256]
print("Batch Size compare:")
print("\tSigmoid vs ReLU\n")
for batch_size in Batch_Sizes:
    print("Batch Size", batch_size)
    model = CNN_model(ac=activation)
    model.compile(optimizer='sgd',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    callback = EarlyStopping(monitor='acc', min_delta=0.001, patience=5)
    bs_logger.append(model.fit(x_train, y_train, batch_size=batch_size, epochs=30, validation_split=0.1, callbacks=[callback]))
    eva = model.evaluate(x_test, y_test)
    print("Test accuracy: {%.2f} loss: {%.2f}" % (eva[1]*100, eva[0]))
    
    K.clear_session()

In [ ]:
color = ['r', 'b', 'g', 'y']
label_n = []

for idx, log in enumerate(bs_logger):
    label_n.append(Batch_Sizes[idx]+'batch acc')
    label_n.append(Batch_Sizes[idx]+'batch val_acc')
    plt.plot(log.history['acc'], '{}.'.format(color[idx]))
    plt.plot(log.history['val_acc'], '{}-'.format(color[idx]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()

label_n = []
for idx, log in enumerate(logger):
    label_n.append(Batch_Sizes[idx]+'batch loss')
    label_n.append(Batch_Sizes[idx]+'batch val_loss')
    plt.plot(log.history['loss'], '{}.'.format(color[idx]))
    plt.plot(log.history['val_loss'], '{}-'.format(color[idx]))

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
activation = 'relu'
opt_logger = []

lr = 0.01
sgd = SGD(lr, momentum=0.0)
sgd_momentum  = SGD(lr, momentum=0.05)
rmsprop = RMSprop(lr)
adam = Adam(lr)

optimizers = [sgd, sgd_momentum, rmsprop, adam]

print("Optimizer compare:")
for opt in optimizers:
    print("Activation Use", activation)
    model = CNN_model(ac=activation)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    callback = EarlyStopping(monitor='acc', min_delta=0.001, patience=5)
    opt_logger.append(model.fit(x_train, y_train, epochs=30, validation_split=0.1, callbacks=[callback]))
    print("Test accuracy: {%.2f} loss: {%.2f}" % (eva[1]*100, eva[0]))

    K.clear_session()

In [ ]:
color = ['r', 'b', 'g', 'y']
opt_name = ['SGD', 'SGD + Momentum', 'RMSprop', 'Adam']
label_n = []

for idx, log in enumerate(opt_logger):
    label_n.append(opt_name[idx]+' acc')
    label_n.append(opt_name[idx]+' val_acc')
    plt.plot(log.history['acc'], '{}.'.format(color[idx]))
    plt.plot(log.history['val_acc'], '{}-'.format(color[idx]))

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()

label_n = []
for idx, log in enumerate(opt_logger):
    label_n.append(opt_name[idx]+' acc')
    label_n.append(opt_name[idx]+' val_acc')
    plt.plot(log.history['loss'], '{}.'.format(color[idx]))
    plt.plot(log.history['val_loss'], '{}-'.format(color[idx]))

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(label_n, loc='upper right')
plt.show()